In [26]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [27]:
model = pyo.ConcreteModel()

In [28]:
model.C = pyo.Var(range(1,4))
model.n = pyo.Var(range(1, 4), within=Integers, bounds=(0, 1000))
model.b = pyo.Var(within= Binary)

In [29]:
C = model.C
n = model.n
b = model.b

In [30]:

model.bj = pyo.Objective(expr = pyo.summation(C))


In [31]:
model.total = pyo.Constraint(expr = pyo.summation(n) == 2100)

model.C1 = pyo.Constraint(expr = C[1] == 2*n[1])

model.C2 = pyo.Constraint(expr = C[2] == b*6*n[2] + 1000)
model.C2n = pyo.Constraint(expr = n[2] <= b*1000)

model.C3 = pyo.Constraint(expr = C[3] == 7*n[3])


In [32]:
model.n.pprint()

n : Size=3, Index=n_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  None :  1000 : False :  True : Integers
      2 :     0 :  None :  1000 : False :  True : Integers
      3 :     0 :  None :  1000 : False :  True : Integers


In [33]:
opt = pyo.SolverFactory('ipopt')
# opt = pyo.SolverFactory('gurobi')
# opt = pyo.SolverFactory('glpk') # -> this will give error , to solve using glpk/cbc u have to convert it into linear
opt.solve(model)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 5, 'Number of variables': 7, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.11.1\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.05305886268615723}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [34]:
print("n1 = ", pyo.value(n[1]))
print("n2 = ", pyo.value(n[2]))
print("n3 = ", pyo.value(n[3]))

print('ntotal =', pyo.summation(n), pyo.value(pyo.summation(n)))

n1 =  1000.0
n2 =  583.3333333377169
n3 =  516.6666566627846
ntotal = n[1] + n[2] + n[3] 2099.9999900005014


In [36]:
print("C1 = ", pyo.value(n[1]))
print("C2 = ", pyo.value(n[2]))
print("C3 = ", pyo.value(n[3]))

print('Ctotal =', pyo.summation(C), pyo.value(pyo.summation(C)))

C1 =  1000.0
C2 =  583.3333333377169
C3 =  516.6666566627846
Ctotal = C[1] + C[2] + C[3] 8658.333283303347


In [37]:
# doing linearization to solve this problenm usikg linear solver

In [38]:
model = pyo.ConcreteModel()

In [39]:
model.C = pyo.Var(range(1,4))
model.n = pyo.Var(range(1, 4), within=Integers, bounds=(0, 1000))
model.b = pyo.Var(within= Binary)

In [40]:
C = model.C
n = model.n
b = model.b

In [41]:
model.bj = pyo.Objective(expr = pyo.summation(C))

In [42]:
M = 100000

In [43]:
model.total = pyo.Constraint(expr = pyo.summation(n) == 2100)

model.C1 = pyo.Constraint(expr = C[1] == 2*n[1])

model.C2a = pyo.Constraint(expr = -b*M <= C[2])
model.C2b = pyo.Constraint(expr = C[2] <= b*M)
model.C2c = pyo.Constraint(expr = -(1-b)*M <= C[2] - (6*n[2]+1000))
model.C2d = pyo.Constraint(expr = C[2] - (6*n[2]+1000) <= (1-b)*M)

model.C2n = pyo.Constraint(expr = n[2] <= b*1000)

model.C3 = pyo.Constraint(expr = C[3] == 7*n[3])


In [44]:
opt = pyo.SolverFactory('glpk')

In [45]:
opt.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 9700.0, 'Upper bound': 9700.0, 'Number of objectives': 1, 'Number of constraints': 8, 'Number of variables': 7, 'Number of nonzeros': 19, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.0506131649017334}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [46]:
print("n1 = ", pyo.value(n[1]))
print("n2 = ", pyo.value(n[2]))
print("n3 = ", pyo.value(n[3]))

print('ntotal =', pyo.summation(n), pyo.value(pyo.summation(n)))

n1 =  1000.0
n2 =  1000.0
n3 =  100.0
ntotal = n[1] + n[2] + n[3] 2100.0


In [47]:
print("C1 = ", pyo.value(n[1]))
print("C2 = ", pyo.value(n[2]))
print("C3 = ", pyo.value(n[3]))70

print('Ctotal =', pyo.summation(C), pyo.value(pyo.summation(C)))

C1 =  1000.0
C2 =  1000.0
C3 =  100.0
Ctotal = C[1] + C[2] + C[3] 9700.0
